# How to use CrewAI Reasoning in Agents

- CrewAI supports agent reasoning to allows agents to reflect on a task and create a plan before execution. 
- This helps the agent approach tasks more methodically and ensures they're ready to perform the assigned work. 

We'll use a simple crew example to demostrate but first let's setup the code we need to run this notebook.

In [1]:
%pip install -U --quiet crewai crewai-tools exa_py
%pip install --quiet --upgrade ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s e

In [ ]:
import warnings

# Filter out deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', message='Pydantic serializer warnings')

### Get the API keys from the environment variable and initialize LLM and ExaSearch Tool

In [ ]:
import os
from crewai import LLM
from crewai_tools import EXASearchTool

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
EXA_API_KEY = os.getenv('EXA_API_KEY')

gemini_model = LLM(model="gemini/gemini-2.5-flash-lite", api_key=GEMINI_API_KEY)
tool = EXASearchTool(api_key=EXA_API_KEY)

In [ ]:
from crewai import Agent, Task, Crew

analyst = Agent(
    role="Research Specialist",
    goal="Conduct detailed research on {topic} using the latest information as of {todays_date}.",
    backstory="You are an expert at web research.",
    llm=gemini_model,
    tools=[tool],
    reasoning=True, # Enable reasoning
    max_reasoning_attempts=3,  # Optional: Set a limit on reasoning attempts
    verbose=True
)

reporter = Agent(
    role="Report Specialist",
    goal="Generate a detailed report on {topic} using the latest information as of {todays_date}.",
    backstory="You are an expert at executive report writing",
    llm=gemini_model,
    verbose=True
)

# Create tasks
analysis_task = Task(
    description="Research and analyze the recent trends and generate a detailed report on {topic}",
    expected_output="A detailed report structured as an executive report for c-suite execs to read.",
    agent=analyst,
    verbose=True
)

report_task = Task(
    description="Generate a detailed report on {topic}",
    expected_output="A detailed report structured as an executive report for c-suite execs to read.",
    agent=analyst,
    markdown=True,
    verbose=True
)

# Create a crew and run the task
crew = Crew(
    agents=[analyst, reporter],
    tasks=[analysis_task, report_task],
    verbose=True
)

result = crew.kickoff(inputs={"topic":"Influencer marketing", "todays_date": "July 31, 2025"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e795263c-f482-490f-97be-9a75799ce827                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: 
datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects 
to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

Output()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: 
datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects 
to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

...

Output()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Specialist                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Influencer Marketing Trends and Strategic Insights for C-Suite Executives (As of July 31, 2025)              │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│                                                                                                                 │
│  Influencer marketing continues its trajectory of robust growth and evolution in 2025, becoming an              │
│  indispensable component of a comprehensive marketing strategy. Driven by shifting platform dynamics,           │
│  advancements in AI, and a persistent consumer demand for authentic engagement, brands are increasingly         │
│  sophisticated in their approach. Key trends highlight the dominance of short-form video content, strategic     │
│  utilization of micro and nano-influencers for niche targeting and trust, and a significant integration of AI   │
│  for campaign optimization, influencer discovery, and performance analytics. Furthermore, there's a heightened  │
│  emphasis on transparent disclosures and ethical practices to maintain consumer trust. ROI measurement is       │
│  evolving, moving beyond vanity metrics to focus on demonstrable business outcomes and sophisticated            │
│  attribution models.                                                                                            │
│                                                                                                                 │
│  ## 1. Current Landscape of Influencer Marketing                                                                │
│                                                                                                                 │
│  As of mid-2025, influencer marketing has solidified its position as a mature and vital marketing channel.      │
│  Brands are allocating substantial budgets, recognizing its unique ability to foster genuine connections,       │
│  build brand credibility, and drive consumer action. The ecosystem is characterized by increased                │
│  professionalism among creators, who often operate as sophisticated businesses, demanding fair compensation,    │
│  clear contracts, and effective collaboration. Consumer trust remains the cornerstone of campaign success,      │
│  directly impacting influencer effectiveness and the perceived authenticity of sponsored content. The industry  │
│  is actively navigating the balance between leveraging technological advancements and preserving the human      │
│  element that underpins influencer appeal.                                                                      │
│                                                                                                                 │
│  ## 2. Key Trends and Innovations in 2025                                                                       │
│                                                                                                                 │
│  *   **Dominance of Short-Form Video:** Platforms like TikTok, Instagram Reels, and YouTube Shorts continue to  │
│  be the primary battlegrounds for consumer attention. I

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c9d9902e-414e-450b-abe9-6cdbd9a46fbb                                                                     │
│  Agent: Research Specialist                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e795263c-f482-490f-97be-9a75799ce827                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Influencer Marketing Trends and Strategic Insights for C-Suite Executives (As of July 31, 2025)              │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│                                                                                                                 │
│  Influencer marketing continues its trajectory of robust growth and evolution in 2025, becoming an              │
│  indispensable component of a comprehensive marketing strategy. Driven by shifting platform dynamics,           │
│  advancements in AI, and a persistent consumer demand for authentic engagement, brands are increasingly         │
│  sophisticated in their approach. Key trends highlight the dominance of short-form video content, strategic     │
│  utilization of micro and nano-influencers for niche targeting and trust, and a significant integration of AI   │
│  for campaign optimization, influencer discovery, and performance analytics. Furthermore, there's a heightened  │
│  emphasis on transparent disclosures and ethical practices to maintain consumer trust. ROI measurement is       │
│  evolving, moving beyond vanity metrics to focus on demonstrable business outcomes and sophisticated            │
│  attribution models.                                                                                            │
│                                                                                                                 │
│  ## 1. Current Landscape of Influencer Marketing                                                                │
│                                                                                                                 │
│  As of mid-2025, influencer marketing has solidified its position as a mature and vital marketing channel.      │
│  Brands are allocating substantial budgets, recognizing its unique ability to foster genuine connections,       │
│  build brand credibility, and drive consumer action. The ecosystem is characterized by increased                │
│  professionalism among creators, who often operate as sophisticated businesses, demanding fair compensation,    │
│  clear contracts, and effective collaboration. Consumer trust remains the cornerstone of campaign success,      │
│  directly impacting influencer effectiveness and the perceived authenticity of sponsored content. The industry  │
│  is actively navigating the balance between leveraging technological advancements and preserving the human      │
│  element that underpins influencer appeal.                                                                      │
│                                                                                                                 │
│  ## 2. Key Trends and Innovations in 2025                                                                       │
│                                                                                                                 │
│  *   **Dominance of Short-Form Video:** Platforms like

### Print the final output

In [5]:
from IPython.display import Markdown

Markdown(result.raw)

```markdown
# Influencer Marketing Trends and Strategic Insights for C-Suite Executives (As of July 31, 2025)

## Executive Summary

Influencer marketing continues its trajectory of robust growth and evolution in 2025, becoming an indispensable component of a comprehensive marketing strategy. Driven by shifting platform dynamics, advancements in AI, and a persistent consumer demand for authentic engagement, brands are increasingly sophisticated in their approach. Key trends highlight the dominance of short-form video content, strategic utilization of micro and nano-influencers for niche targeting and trust, and a significant integration of AI for campaign optimization, influencer discovery, and performance analytics. Furthermore, there's a heightened emphasis on transparent disclosures and ethical practices to maintain consumer trust. ROI measurement is evolving, moving beyond vanity metrics to focus on demonstrable business outcomes and sophisticated attribution models.

## 1. Current Landscape of Influencer Marketing

As of mid-2025, influencer marketing has solidified its position as a mature and vital marketing channel. Brands are allocating substantial budgets, recognizing its unique ability to foster genuine connections, build brand credibility, and drive consumer action. The ecosystem is characterized by increased professionalism among creators, who often operate as sophisticated businesses, demanding fair compensation, clear contracts, and effective collaboration. Consumer trust remains the cornerstone of campaign success, directly impacting influencer effectiveness and the perceived authenticity of sponsored content. The industry is actively navigating the balance between leveraging technological advancements and preserving the human element that underpins influencer appeal.

## 2. Key Trends and Innovations in 2025

*   **Dominance of Short-Form Video:** Platforms like TikTok, Instagram Reels, and YouTube Shorts continue to be the primary battlegrounds for consumer attention. Influencers who excel in creating engaging, concise, and entertaining short-form video content are highly sought after. This format is favored for its high engagement rates, discoverability, and potential for virality.
*   **Strategic Tiering of Influencers:**
    *   **Nano and Micro-Influencers:** These creators, with smaller but highly engaged and niche audiences, are proving invaluable for building trust and driving conversion within specific communities. Brands are increasingly prioritizing them for their authenticity and higher engagement rates relative to follower count. Reports indicate that micro-influencers are increasingly favored for delivering top ROI in 2025.
    *   **Macro and Mega-Influencers:** While still relevant for broad brand awareness and large-scale campaigns, their effectiveness is increasingly scrutinized for genuine audience connection versus superficial reach. Their use is often more targeted and integrated with other marketing efforts.
*   **The Rise of AI in Influencer Marketing:** Artificial Intelligence is no longer a futuristic concept but a tangible force reshaping the industry:
    *   **AI-Powered Discovery and Vetting:** AI tools are enhancing the ability to identify suitable influencers by analyzing audience demographics, engagement patterns, content relevance, and detecting inauthentic activity, thus improving campaign efficiency and mitigating risks.
    *   **AI in Content Creation and Optimization:** AI assists in generating content ideas, optimizing content for specific platforms, and even creating AI-generated virtual influencers (avatars). While virtual influencers offer creative control and unique brand alignment, their ethical use and impact on authenticity are subjects of ongoing discussion.
    *   **Advanced Analytics:** AI provides deeper insights into campaign performance, audience sentiment, and predictive modeling for future campaign success.
*   **Emphasis on Authenticity and Storytelling:** Consumers are increasingly discerning and wary of overly polished or inauthentic endorsements. Influencers who can weave compelling narratives, share genuine experiences, and maintain a relatable persona, even within sponsored content, achieve significantly higher audience engagement and trust. Transparency in partnerships is a key ethical consideration in 2025.
*   **Maturation of the Creator Economy:** The professionalization of creators continues, with many establishing robust personal brands and businesses. This leads to a greater demand for fair compensation, clear performance expectations, and collaborative partnerships that respect the creator's voice and audience relationship.

## 3. Measurement, ROI, and Attribution

*   **Shift Towards Performance Metrics:** The industry is moving beyond traditional engagement metrics (likes, comments, shares) to focus on tangible business outcomes. Key metrics now include:
    *   **Conversion Rates:** Tracking how many users take a desired action (e.g., purchase, sign-up) after interacting with influencer content.
    *   **Customer Acquisition Cost (CAC):** Measuring the efficiency of influencer campaigns in acquiring new customers.
    *   **Brand Lift:** Assessing the impact on brand awareness, perception, and purchase intent through surveys and analytics.
    *   **Website Traffic & Leads:** Monitoring referral traffic and lead generation directly attributable to influencer efforts.
    *   **ROI:** Maximizing ROI remains a primary goal, with a focus on demonstrating measurable returns.
*   **Sophisticated Attribution Models:** Brands are employing more advanced methods to track the customer journey and attribute value accurately:
    *   **UTM Parameters & Unique Discount Codes:** Standard practices for tracking traffic sources and direct sales.
    *   **Affiliate Marketing:** Leveraging affiliate links to incentivize sales and track commissions.
    *   **Advanced Analytics & CRM Integration:** Utilizing platforms that can connect influencer touchpoints to final conversions, often integrating with CRM systems.
*   **Data-Driven Decision Making:** The reliance on data analytics is paramount for understanding influencer performance, optimizing campaign spend, and demonstrating ROI to stakeholders.

## 4. Challenges and Opportunities

*   **Challenges:**
    *   **Maintaining Authenticity and Trust:** Countering the perception of saturation and inauthenticity by partnering with genuine influencers and ensuring transparent disclosures. Ethical considerations are paramount.
    *   **Navigating Regulatory Compliance:** Adhering to evolving FTC guidelines and disclosure mandates to avoid legal repercussions and protect brand reputation.
    *   **Accurate ROI Measurement:** Quantifying the precise business impact and attributing sales effectively remains a complex but critical objective.
    *   **Combating Fraud:** Vigilance against fake followers, engagement pods, and deceptive practices continues to be a priority.
*   **Opportunities:**
    *   **Strategic AI Integration:** Leveraging AI for enhanced influencer discovery, content personalization, and performance insights can drive significant efficiency and effectiveness.
    *   **Niche Community Engagement:** Tapping into highly specific and passionate communities through specialized nano and micro-influencers offers targeted impact.
    *   **Long-Term Influencer Partnerships:** Cultivating enduring relationships with influencers can lead to deeper brand advocacy and more authentic audience connections over time.
    *   **Cross-Platform Synergy:** Developing integrated influencer strategies that span multiple platforms to create a cohesive and amplified brand message.
    *   **Ethical Leadership:** Brands that champion transparency, fair creator compensation, and authentic partnerships can build stronger, more loyal customer bases. The future of influencer marketing in 2025 is closely tied to ethical practices.

## 5. Strategic Recommendations for C-Suite Executives

*   **Prioritize Authenticity and Transparency:** Ensure all influencer collaborations are transparent, with clear disclosures. Partner with influencers whose values and audience align genuinely with your brand. Ethical considerations are non-negotiable.
*   **Embrace Data-Driven Strategy:** Invest in robust analytics tools and methodologies to measure campaign performance rigorously. Focus on metrics that directly contribute to business objectives (e.g., conversion, ROI) rather than vanity metrics alone.
*   **Diversify Your Influencer Portfolio:** Employ a balanced approach to influencer tiers. Utilize nano and micro-influencers for deep engagement and niche targeting, and macro/mega-influencers strategically for broader reach and impact.
*   **Integrate AI Thoughtfully:** Explore and adopt AI-powered tools for influencer discovery, audience analysis, and campaign optimization. However, always ensure the human element of creativity, connection, and authenticity is preserved.
*   **Stay Vigilant on Regulations and Ethics:** Maintain an up-to-date understanding of influencer marketing regulations (e.g., FTC guidelines) and ensure all campaigns are fully compliant. Ethical marketing practices are key to long-term success.
*   **Foster Strategic Partnerships:** Move beyond transactional relationships to build collaborative, long-term partnerships with influencers who can become true brand advocates. This fosters greater authenticity and sustained engagement.
*   **Allocate Budget Wisely:** Continue to allocate significant resources to influencer marketing, but ensure strategic allocation based on performance data and clearly defined campaign goals.

The influencer marketing landscape in 2025 demands a strategic, data-informed, and ethically grounded approach. By embracing these trends and recommendations, brands can harness the full potential of influencer marketing to drive meaningful business growth and build lasting connections with their target audiences.
```

### Conclusion

If you inspect the task outputs above, you'll see that there's traces of agent reasoning over the task first before proceeding to accomplish it.

Visit our docs for more details on [CrewAI Reasoning](https://docs.crewai.com/en/concepts/reasoning).

Happy building! 🚀